In [ ]:
%run "0_Workspace_setup.ipynb"

from NHM_helpers.output_plots import create_streamflow_plot
from NHM_helpers.NHM_hydrofabric import make_hf_map_elements 
from NHM_helpers.map_template import make_streamflow_map
from NHM_helpers.NHM_output_visualization import retrieve_hru_output_info
from ipywidgets import widgets
from IPython.display import display
import webbrowser
poi_id_sel = None
crs = 4326

## Introduction
This notebook maps NHM domain gages with KGE values and displays in a new browser tab. A gage id can be selected and a plot created that shows a time-series of simulated and observed streamflow at daily, monhtly, and annual timesteps for the selected gage. A flow exceedance curve and table of summary statistics is also provided in the plot for streamflow evaluation purposes. Additionally, maps and plots produced are saved for use outside of notebooks as .html files in the `html_maps` and `html_plots` folders in the `notebook_output` folder.

In [ ]:
(
    hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    gages_df,
    gages_txt,
    gages_txt_nb2,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    default_gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)
# Retrieve pywatershed output file information for mapping and plotting
plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)

con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}",
)

## Create an interactive map to evaluate streamflow at gages (pois)

In [ ]:
map_file = make_streamflow_map(
    out_dir,
    plot_start_date,
    plot_end_date,
    water_years,
    hru_gdf,
    poi_df,
    poi_id_sel,
    seg_gdf,
    nwis_gages_aoi,
    gages_df,
    html_maps_dir,
    param_filename,
    subbasin,
    HW_basins_gdf,
    HW_basins,
    output_netcdf_filename,
)

map_file_str = f"{map_file}"
_ = webbrowser.open(map_file_str, new=2)

## Paste the selected gage id in the field below

#### &#x270D;<font color='green'>**Enter Information:**</font> **Select a gage id and make a streamflow plot.**
> The default is set to the first gage in the `poi_df`. If a gage number is selected here,
> the map will zoom to that location if redrawn.

In [ ]:
v = widgets.Combobox(
    # value='John',
    placeholder="(optional) Enter Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
)


def on_change(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v.value


v.observe(on_change)

display(v)

#### &#x1F6D1;If a new selection is made above,
><font color='green'>**select this cell**</font>, then select <font color='green'>**Run Selected Cell and All Below**</font> from the Run menu in the toolbar.

In [ ]:
# Make plot
# This is for testing only; can comment out in user version
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]

plot_file_path = create_streamflow_plot(
    poi_id_sel,
    plot_start_date,
    plot_end_date,
    water_years,
    html_plots_dir,
    output_netcdf_filename,
    out_dir,
)